# Ensemble: Boosting

- 다양한 모델을 결합하여 예측 성능을 향상시키는 방법
    - 깊이가 얕은 결정트리를 사용해 이전 트리의 오차를 보정하는 방식
    - 순차적으로 경사하강법을 사용해 이전 트리의 오차를 줄여나감
        - 분류 모델에서는 손실함수 LogLoss를 사용해 오차를 줄임
        - 회귀 모델에서는 손실함수 MSE를 사용해 오차를 줄임
    - Boosting 계열은 일반적으로 결정트리 개수를 늘려도 과적합에 강함
    - 대표 알고리즘(모델): GradientBoosting, HistGradientBoosting,  **XGBoost (DMLC)**,  LightGBM(MS), CatBoost

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  

## GradientBoosting 구현

In [1]:
from sklearn.tree import DecisionTreeRegressor

In [5]:
class SimpleGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate = 0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.initial_log_odds = 0       # 초기 예측값
        self.trees = []                 # 내부적으로 가지고 있는 estimator 모음 배열
    
    
    def log_odds(self, p): # p: 확률값
        # 확률값 -> 로짓변환 (컴큐터 더 편하게 할라고)
        return np.log(p / (1-p)) 
    
    
    def sigmoid(self, z):
        # z값 -> 0~1 사이의 확률 값 변환
        return 1 /(1+np.exp(-z))
    
    
    def fit(self, X, y):
        y_mean = np.mean(y) # 초기 예측값: 평균 (0과 1 다 더한 평균)
        self.initial_log_odds = self.log_odds(y_mean)
        y_pred_log_odds = np.full_like(y, self.initial_log_odds, dtype=np.float64)
        
        for _ in range(self.n_estimators): 
            y_pred_proba  = self.sigmoid(y_pred_log_odds)
            residual = y - y_pred_proba # 이 잔차 값으로 학습을 진행함
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred_log_odds += self.learning_rate * tree.predict(X)
    
    
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int) # boolean을 0 혹은 1로 변환
        
    
    def predict_proba(self, X):
        y_pred_log_odds = np.full((X.shape[0],), self.initial_log_odds) # 초기값 세팅
        
        for tree in self.trees:
            y_pred_log_odds += self.learning_rate * tree.predict(X)
        
        return self.sigmoid(y_pred_log_odds)

In [12]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)

simple_gb_clf = SimpleGradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=3
)

simple_gb_clf.fit(X_train, y_train)

y_pred_train = simple_gb_clf.predict(X_train)
y_pred_test = simple_gb_clf.predict(X_test)

print(f"학습 정확도: {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도: {accuracy_score(y_test, y_pred_test)}")

학습 정확도: 1.0
평가 정확도: 0.965034965034965
